In [48]:
!pip install base58
!pip install ecdsa
!pip install prettytable


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [67]:
!pip install bitcoin

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for bitcoin: filename=bitcoin-1.1.42-py3-none-any.whl size=44414 sha256=93eedc681858772e6020ea8302112e250ad5854e2f5a949fc6485bad1c456957
  Stored in directory: c:\users\nasyk\appdata\local\pip\cache\wheels\a3\98\a1\0d53f5b714680bb5dffc2db778c221b1f5c54bc4a3a779c8c5
Successfully built bitcoin



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
import hashlib
import base58
import binascii
import os
import ecdsa
from ecdsa import SigningKey, SECP256k1
from prettytable import PrettyTable

### Prerequisite

1. Generate two testnet addresses: Address_1, Address_2

Helping code from seminar notebook

In [64]:
def hashing(a):
    first_sha256 = hashlib.sha256(a)
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return_1 = bytes.fromhex("00") + ripemd160.digest()
    return_2 = bytes.fromhex("6f") + ripemd160.digest()
    return return_0,return_1, return_2
    #return bytes.fromhex("34") + ripemd160.digest()

In [40]:
def checksum(b):
    checksum_full = hashlib.sha256(hashlib.sha256(b).digest()).digest()
    new_checksum = checksum_full[:4]
    return b + new_checksum

In [41]:
def to_base58(c):
    return base58.b58encode(c).decode('utf-8')

In [42]:
def compressed_key(d):
    a = d.hex()

    if a[-1] == '0' or a[-1] == '2' or a[-1] == '4' or a[-1] == '6' or a[-1] == '8' or a[-1] == 'a' or a[-1] == 'c' or a[-1] == 'e':
        return bytes.fromhex("02") + d
    
    else:
        return bytes.fromhex("03") + d

In [43]:
def generate_address():
    
    private_key = os.urandom(32).hex()
    private_key_from_hex = bytes.fromhex(private_key)

    sk = ecdsa.SigningKey.from_string(private_key_from_hex, curve = ecdsa.SECP256k1) 
    verification_key = sk.verifying_key

    x = verification_key.to_string().hex()[:64]
    y = verification_key.to_string().hex()[64:]

    public_key = bytes.fromhex("04") + verification_key.to_string()

    compressed_public_key = compressed_key(verification_key.to_string())
    compressed_public_key_hex = compressed_public_key.hex()[:66]

    decoded_pubkey, mainnet_pubkey, testnet_pubkey = hashing(public_key)
    
    checksum_main_pubkey = checksum(mainnet_pubkey)
    main_address = to_base58(checksum_main_pubkey)

    checksum_test_pubkey = checksum(testnet_pubkey)
    test_address = to_base58(checksum_test_pubkey)
    
    print(f'''
________________NEW DATA________________
Private Key: {private_key} 
Public Key: {public_key.hex()} 
Compressed_Key: {compressed_public_key_hex} 
MainNet Address: {main_address} 
Testnet Address: {test_address}
    ''')
    
    return private_key, public_key, compressed_public_key_hex, main_address, test_address

In [65]:
# Address1
private_key1, public_key1, compressed_public_key1_hex, main_address1, test_address1 = generate_address()


________________NEW DATA________________
Private Key: e9e0825fa7d83dbc2ffd32166c2bf2cb7e77b38d8eab269cc51958e8b91c221e 
Public Key: 04847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a28 
Compressed_Key: 02847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada6 
MainNet Address: 1B4hpYcokLDCyfVR9THVPXjSU9ufifbMPY 
Testnet Address: mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R
    


In [66]:
# Address2
private_key2, public_key2, compressed_public_key2_hex, main_address2, test_address2 = generate_address()


________________NEW DATA________________
Private Key: e777808d97db3d835b5ada8b9e9a0661dda6e96d0fdb64f02a8e9642603ddc7c 
Public Key: 04b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea92 
Compressed_Key: 02b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f 
MainNet Address: 1BE4yv7hWLnK825pxdWTAvTtyZbnihvmqQ 
Testnet Address: mqk2GyCgKNDZu8ZSgCUpzqgDqZCVZtzW4R
    


2. Using address_1 and address_2 creates a multi-signature address.

In [80]:
import bitcoin 

def public_to_test(a):
    decoded_address = base58.b58decode(scriptaddress)
    address_1 = decoded_address[1:-4]
    hex_address = bytes.fromhex("C4") + address_1
    checksum_test_multsig = checksum(hex_address)
    test_address = to_base58(checksum_test_multsig)
    return test_address

In [81]:
print("Generating MultiSig Address and Test Address M:N")
m = int(input("Enter N (Max), for example, 2: "))
n = int(input("Enter M (Min), for example, 2: "))

pub_keys = []
for i in range(0,m):
    if i == 1:
        temp = input("Enter {} of {} public key, for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: ".format(i+1,m))
    else:
        temp = input("Enter {} of {} public key, for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: ".format(i+1,m))
    pub_keys.append(temp)
    
rawscript = bitcoin.mk_multisig_script(pub_keys,n)
descript = bitcoin.deserialize_script(rawscript)
scriptaddress = bitcoin.scriptaddr(rawscript)
print("MultiSig Address: ", scriptaddress)

multisig_testnet_address = public_to_test(scriptaddress)
print("MultiSig Testnet Address: ", multisig_testnet_address)

print("\nRedeem Script:", rawscript)

Generating MultiSig Address and Test Address M:N
Enter N (Max), for example, 2: 2
Enter M (Min), for example, 2: 2
Enter 1 of 2 public key, for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: 04847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a28
Enter 2 of 2 public key, for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: 04b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea92
MultiSig Address:  3MuuCJgZoYM4CHGjwVzNUhCnGEyN8tkXca
MultiSig Testnet Address:  2NDU7G3cbQzrQQ4uHcdcF6eC3UbBXuBVjZa

Redeem Script: 524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b274

### Task 1 (1 point)

Provide addresses

a. Address_1 and Address_2

Testnet Address_1: mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R

Testnet Address_2: mqk2GyCgKNDZu8ZSgCUpzqgDqZCVZtzW4R

b. MultiSignature Address and Redeem Script

MultiSignature Testnet Address: 2NDU7G3cbQzrQQ4uHcdcF6eC3UbBXuBVjZa

Redeem Script: 524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea9252ae

### Task 2 (1 point)

Get some testnet Bitcoins to your testnet address_1 from any source.
Provide the Transaction_ID.

(Best source: https://coinfaucet.eu/en/btc-testnet/).


Address: https://live.blockcypher.com/btc-testnet/address/mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R/

TX: cccbbd5526f1f09ab09b2254e180971730ae79204092d260a79bcc0c658d7d29

https://live.blockcypher.com/btc-testnet/tx/cccbbd5526f1f09ab09b2254e180971730ae79204092d260a79bcc0c658d7d29/

### Task 3 (3 points)

Create a new transaction from address_1 and provide Transaction_ID.
Make two outputs:

a. Some bitcoins to address_2.

b. Some bitcoins to Multi_Signature Address.

Helping code from seminar notebook

In [82]:
class variableInteger():    
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)

    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b

    def variable_integer(a):
        temp = int(len(a)/2)
        if temp <= int('0xfc',16):
            return hex(temp)[2:]
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  reverse_string_in_pair(padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + reverse_string_in_pair(padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + reverse_string_in_pair(padding(hex(temp)[2:]))

In [83]:
def bytewise_reverse(hex_string):
    i = 0
    reverted_string = []
    while i != len(hex_string):
        temp_value = hex_string[i] + hex_string[i+1]
        reverted_string.append(temp_value)
        i = i + 2
    reverted_string.reverse()
    return ''.join(reverted_string)

def decode_raw_transaction(raw_transaction):
    print('''
===============================
Decode raw transaction''')
    print(raw_transaction)
    print('''
================================''')
    print('''
***Values are in reverse byte format!***''')
    version_number_string = raw_transaction[:8]
    print("Version number, 4 bytes: ", 
          version_number_string, 
          " (",
          int(bytewise_reverse(version_number_string), 16), 
          ")")
    print("Flag (optional), 0 or 2 bytes: Empty") # only Empty is supported in this function
    print('''
****************************
***Input info starts here***
****************************''')
    number_of_inputs_string = raw_transaction[8:10]
    number_of_inputs = int(number_of_inputs_string, 16)
    print("Number of inputs, variable length integer 1-9 bytes: ", number_of_inputs_string, 
          " (",
          number_of_inputs, 
          ")") # only < 0xFD is supported in this function
    next_symbol_to_read = 10
    for input_index in range(number_of_inputs):
        print("***Input_" + str(input_index), "***")
        print("Previous output tx hash, 32 bytes: ", raw_transaction[next_symbol_to_read:next_symbol_to_read + 64],
             " (",
              bytewise_reverse(raw_transaction[next_symbol_to_read:next_symbol_to_read + 64]), 
              ")")
        next_symbol_to_read = next_symbol_to_read + 64
        print("Previous tx output index, 4 bytes: ", raw_transaction[next_symbol_to_read:next_symbol_to_read + 8],
             " (",
              int(bytewise_reverse(raw_transaction[next_symbol_to_read:next_symbol_to_read + 8]), 16), 
              ")")
        next_symbol_to_read = next_symbol_to_read + 8
        current_signature_size_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + 2] # only <0xFD is supported
        current_signature_size = int(bytewise_reverse(current_signature_size_string), 16)
        print("Sig script size in bytes, variable length integer 1-9 bytes: ", current_signature_size_string,
              " (",
              current_signature_size,
              ")")
        next_symbol_to_read = next_symbol_to_read + 2
        sig_script_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + current_signature_size * 2]
        print("Sig script: ", sig_script_string)
        next_symbol_to_read = next_symbol_to_read + current_signature_size * 2
        print("Sequence number, 4 bytes: ffffffff")
        next_symbol_to_read = next_symbol_to_read + 8
    print('''
****************************
***Output info starts here***
****************************''')
    number_of_outputs_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + 2]
    number_of_outputs = int(number_of_outputs_string, 16)
    print("Number of outputs, variable length integer 1-9 bytes: ", 
          number_of_outputs_string, 
          " (",
          number_of_outputs, 
          ")") # only 1 byte is supported
    next_symbol_to_read = next_symbol_to_read + 2
    for output_index in range(number_of_outputs):
        print("***Output_" + str(output_index), "***")
        amount_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + 16]
        print("Amount in satoshi, 8 bytes: ", 
              amount_string, 
              "( ", 
              int(bytewise_reverse(amount_string), 16), 
              " )")
        next_symbol_to_read = next_symbol_to_read + 16
        output_script_size_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + 2] # only <0xFD is supported
        output_script_size = int(bytewise_reverse(output_script_size_string), 16)
        print("Output script size in bytes, variable length integer 1-9 bytes: ", output_script_size_string,
              " (",
              output_script_size,
              ")")
        next_symbol_to_read = next_symbol_to_read + 2
        output_script_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + output_script_size * 2]
        print("Script hash: ", 
              output_script_string)
        next_symbol_to_read = next_symbol_to_read + output_script_size * 2
    print('''
****************************''')
    locktime_string = raw_transaction[next_symbol_to_read:next_symbol_to_read + 8]
    locktime = int(bytewise_reverse(locktime_string), 16)
    print("Locktime, 4 bytes: ",  locktime_string, "( ", locktime, " )")
    next_symbol_to_read = next_symbol_to_read + 8

In [84]:
def doublehash256_txid(a): 
   temp_bin = binascii.unhexlify(a)
   temp_hash = hashlib.sha256(temp_bin).digest()
   hash2 = hashlib.sha256(temp_hash).digest()
   return str(binascii.hexlify(hash2),"ascii")

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = bytewise_reverse(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = bytewise_reverse(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = bytewise_reverse('{:08x}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index + place_holder() +  'ffffffff'

def place_holder():
    previous_place_holder = input('''Enter Output Script 
    OP_HASH160(public_key)=RIPEMD160(SHA256(public_key)): ''')
    size_receiver_1 = hex(int(len(previous_place_holder)/2))[2:]
    script_Pubkey_prev_op = '76' + 'a9' + size_receiver_1 + previous_place_holder + '88' + 'ac'
    script_len_1 = hex(int(len(script_Pubkey_prev_op)/2))[2:]
    return script_len_1 + script_Pubkey_prev_op

def tx_outputs(c):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
def generate_raw_transaction(print_output = True):
    #Creating Raw Transaction for Double Hash
    #P2PKH
    version = 1
    version = bytewise_reverse('{:08d}'.format(int(version)))

    #_________________TALKING PREVIOUS TRANSACTIONS_________________
    utxo_ip = '01'
    prev_tx_datas = utxo_ip + tx_inputs_tx_id()


    #___________________________________________________________________
    utxo_output = input("Enter number of outputs, for example, 2: ")
    tx_output = ''
    for i in range(0, int(utxo_output)):
            tx_output = tx_outputs(i) + tx_output
    next_output_data = '0' + utxo_output + tx_output
    #____________________________________________________________________________

    locktime = '00000000'
    SigHashCode = '01000000'


    #Creating Raw transaction:
    print(
    """
    Raw_Transaction for 1 input Data and {} Output
    """.format(tx_output))
    #Printing Input PreSignature:
    
    raw_transaction = version + prev_tx_datas + next_output_data + locktime + SigHashCode
    if print_output:
        print(f'''
        Version {version}
        Previous data: {prev_tx_datas}
        Next output data: {next_output_data} 
        Locktime: {locktime} 
        SigHashCode: {SigHashCode}
        ''')
        print(raw_transaction)
    return(raw_transaction)    

In [85]:
def sign_raw_transaction(print_output = True):
    unsigned = input("Enter Raw data Transaction: ")

    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    hash2 = hashlib.sha256(temp_hash).hexdigest()
    tx_hash = hashlib.sha256(temp_hash).digest()

    private_key = input("Enter Signing Private Key: ")
    signingKey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingKey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)
    SIG = str(binascii.hexlify(SIG),'ascii')
    if print_output:
        print("\n\nSig Script: ", SIG)
    return(SIG)

In [86]:
# Exploring Signature
def decode_digital_signature():
    #_______Digital Signature Decoder__________
    DER_input =  input("Enter Digital Signature, for example, 3045022100cd89b747b9507cb525c5d2d48aa7ebc8941c257621219471dab1e23c6280f99502204823508b23dc61e2868c120f1b90da864902266a3d2ae087720d26e5313d788a: ")
    print("\n")

    #Variables:
    header = DER_input[0:2]
    Sig_Length = DER_input[2:4]
    r_integer = DER_input[4:6]
    r_length = DER_input[6:8]
    r = DER_input[8:8 + 2 * int(r_length,16)]
    s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
    s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
    s = DER_input[12 + 2 * int(r_length,16):]

    print("\033[4mDECODING DIGITAL SIGNATURE\033[0m\n")
    x = PrettyTable()
    x.field_names = ["Fields","Value (in Hex)"]
    x.add_row(["Header",header])
    x.add_row(["Sig Length",Sig_Length])
    x.add_row(["Integer",r_integer])
    x.add_row(["R Length",r_length])
    x.add_row(["R",r])
    x.add_row(["Integer",s_integer])
    x.add_row(["S Length",s_length])
    x.add_row(["S",s])
    print(x)


    #_____DER_SIG_SCRIPT_ENCODER____________
    def pushdata(len_r, len_s):
        if len_r == '20' and len_s == '20':
            return str(47)    
        elif len_r == '20' or len_s == '21':
            return str(48)
        elif len_r == '21' or len_s == '20':
            return str(48)
        else:
            return str(49)

    def pubkey_opcode(a):
        return hex(int(int(len(a))/2))[2:]



    pushdata_opcode = pushdata(r_length, s_length)
    sighash_code = '01'

    publickey = input("Enter Publickey (Compressed or Uncompressed), for example, 049ea82ac663a76fb872d4a49674762c3f9e60e25d55f7636d76b03efef3d69e838ab749455d3951aa015ccac729ecd9d656e596686af6204534821699e779534b: ")
    pubkey_pushdata_opcode = pubkey_opcode(publickey)

    y = PrettyTable()

    print("\033[4mSIG SCRIPT\033[0m\n")
    y.field_names = ["Fields","Value (in Hex)"]
    y.add_row(["PUSHDATA Opcode",pushdata_opcode])
    y.add_row(["Header",header])
    y.add_row(["Sig Length",Sig_Length])
    y.add_row(["Integer",r_integer])
    y.add_row(["R Length",r_length])
    y.add_row(["R",r])
    y.add_row(["Integer",s_integer])
    y.add_row(["S Length",s_length])
    y.add_row(["S",s])
    y.add_row(["------","------"])
    y.add_row(["SigHash Code",sighash_code])
    y.add_row(["PUSHDATA Opcode",pubkey_pushdata_opcode])
    y.add_row(["Public Key",publickey])

    print(y)

    final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
    z = PrettyTable()
    z.field_names = ["Fields","Value (in Hex)"]
    z.add_row(["SigScript Lenght",pushdata_opcode])
    z.add_row(["Final SigScript",final_SigScript])

    print(z)
    print("\nSigScript(hex): ",pubkey_opcode(final_SigScript) + final_SigScript,"\n\n\n")

In [89]:
def hashing(a):
    first_sha256 = hashlib.sha256(bytes.fromhex(a))
    ripemd160 = hashlib.new("ripemd160")
    ripemd160.update(first_sha256.digest())
    return_0 = ripemd160.digest()
    return return_0.hex()

In [92]:
print('Hash_160:', hashing(public_key1.hex()))

Hash_160: 6e6494907268b2df0ad95095fc69b36187022fa6


In [94]:
raw_transaction = generate_raw_transaction()

Enter Reference TxId: cccbbd5526f1f09ab09b2254e180971730ae79204092d260a79bcc0c658d7d29
Enter Previous output index: 1
Enter Output Script 
    OP_HASH160(public_key)=RIPEMD160(SHA256(public_key)): 6e6494907268b2df0ad95095fc69b36187022fa6
Enter number of outputs, for example, 2: 2
Enter Receiver's Address 1: mqk2GyCgKNDZu8ZSgCUpzqgDqZCVZtzW4R
Enter Value in BTC (0.0) to send mqk2GyCgKNDZu8ZSgCUpzqgDqZCVZtzW4R:0.00001
Enter Receiver's Address 2: 2NDU7G3cbQzrQQ4uHcdcF6eC3UbBXuBVjZa
Enter Value in BTC (0.0) to send 2NDU7G3cbQzrQQ4uHcdcF6eC3UbBXuBVjZa:0.00001

    Raw_Transaction for 1 input Data and e80300000000000017a914ddd24e8582c1516fd3d2f87a66df7e291136139c87e8030000000000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588ac Output
    

        Version 01000000
        Previous data: 01297d8d650ccc9ba760d292402079ae30179780e154229bb09af0f12655bdcbcc010000001976a9146e6494907268b2df0ad95095fc69b36187022fa688acffffffff
        Next output data: 02e80300000000000017a914ddd24e8582c1516fd3

In [95]:
decode_raw_transaction(raw_transaction)


Decode raw transaction
0100000001297d8d650ccc9ba760d292402079ae30179780e154229bb09af0f12655bdcbcc010000001976a9146e6494907268b2df0ad95095fc69b36187022fa688acffffffff02e80300000000000017a914ddd24e8582c1516fd3d2f87a66df7e291136139c87e8030000000000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588ac0000000001000000


***Values are in reverse byte format!***
Version number, 4 bytes:  01000000  ( 1 )
Flag (optional), 0 or 2 bytes: Empty

****************************
***Input info starts here***
****************************
Number of inputs, variable length integer 1-9 bytes:  01  ( 1 )
***Input_0 ***
Previous output tx hash, 32 bytes:  297d8d650ccc9ba760d292402079ae30179780e154229bb09af0f12655bdcbcc  ( cccbbd5526f1f09ab09b2254e180971730ae79204092d260a79bcc0c658d7d29 )
Previous tx output index, 4 bytes:  01000000  ( 1 )
Sig script size in bytes, variable length integer 1-9 bytes:  19  ( 25 )
Sig script:  76a9146e6494907268b2df0ad95095fc69b36187022fa688ac
Sequence number, 4 bytes: ffffffff

In [96]:
signature = sign_raw_transaction()

Enter Raw data Transaction: 0100000001297d8d650ccc9ba760d292402079ae30179780e154229bb09af0f12655bdcbcc010000001976a9146e6494907268b2df0ad95095fc69b36187022fa688acffffffff02e80300000000000017a914ddd24e8582c1516fd3d2f87a66df7e291136139c87e8030000000000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588ac0000000001000000
Enter Signing Private Key: e9e0825fa7d83dbc2ffd32166c2bf2cb7e77b38d8eab269cc51958e8b91c221e


Sig Script:  30440220396d14556ca75f4c81189276291d16d72593bbc369f2bda8e66712fe8d2f1607022013131fc6d89dc1a47421831116c61533c180fa4f05a1f01e7193c2d43955eef2


In [97]:
decode_digital_signature()

Enter Digital Signature, for example, 3045022100cd89b747b9507cb525c5d2d48aa7ebc8941c257621219471dab1e23c6280f99502204823508b23dc61e2868c120f1b90da864902266a3d2ae087720d26e5313d788a: 30440220396d14556ca75f4c81189276291d16d72593bbc369f2bda8e66712fe8d2f1607022013131fc6d89dc1a47421831116c61533c180fa4f05a1f01e7193c2d43955eef2


DECODING DIGITAL SIGNATURE

+------------+------------------------------------------------------------------+
|   Fields   |                          Value (in Hex)                          |
+------------+------------------------------------------------------------------+
|   Header   |                                30                                |
| Sig Length |                                44                                |
|  Integer   |                                02                                |
|  R Length  |                                20                                |
|     R      | 396d14556ca75f4c81189276291d16d72593bbc369f2bda8e66712fe8d

Raw transaction

01000000

01

297d8d650ccc9ba760d292402079ae30179780e154229bb09af0f12655bdcbcc

01000000

1976a9146e6494907268b2df0ad95095fc69b36187022fa688ac

ffffffff

02

e803000000000000

17a914ddd24e8582c1516fd3d2f87a66df7e291136139c87e8030000000000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588ac

00000000

01000000

0100000001df32efd09c04b00410d307e0c0b2fb63ec0d298167990540bb13fde01fafb81b010000008a473044022031be4eee90a74b8df11e73890f9a026b279d748e265ffe64d39025bfa0e1ddf102205bed2f7438282a7de0ec9c5e1b6a74267edf1fc9d6cb2e56ead773b9872ac6f8014104ae30407ca07e0b6c6fce928b8ddc3a8963683ca1754db19b582757ed8964bd65c79e9c888f2e61c69fd151dba4b7838fb3e6ed6f09b4ff1ffceec952a72ae73bffffffff012f750000000000001976a91489e3df90a6c6d6ba7a4780d01b4fc2f9e0f8d06b88ac00000000

TX: fd3188717654258f8ec3b1a85292fe46e71548936afbd967d5a8b397b60f1092

https://live.blockcypher.com/btc-testnet/tx/fd3188717654258f8ec3b1a85292fe46e71548936afbd967d5a8b397b60f1092/

### Task 4 (2 points)

From Step 3.1, Make a Null Transaction/Data Transfer to self (address_1) with your Lastname FirstName.

In [98]:
def reverse(a):
    i = 0
    rev_a = []
    while i != len(a):
        temp_value = a[i] + a[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

In [99]:
def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse(str(temp_1).zfill(16))
    return temp_2

In [100]:
def bytes_padding(temp_no):
    if len(temp_no)/2 != 0:
        return '0' + temp_no
    else:
        return temp_no

In [101]:
def tx_outputs():
    receiver = input("Enter Receiver's Address {}, for example, mp7cUPuKFeR3tm7hMF6Y9PZX89iyHMK67E: ".format(1))
    value = input("Enter Value in BTC (0.0) to send {}, for example, 0.009:".format(receiver))
    value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
    receiver_address = base58.b58decode(receiver).hex()[2:-8]
    size_receiver = hex(int(len(receiver_address)/2))[2:]
    locking_Script = '76' + 'a9' + size_receiver + receiver_address + '88' + 'ac'
    script_len_2 = hex(int(len(locking_Script)/2))[2:]
    return value_in_hex_NYB + script_len_2 + locking_Script

In [102]:
def op_return():
    value_return = '0'*16
    #data_1 = b"charley loves heidi"
    data = input("Enter Data to send, for exampe, 'Hello': ")
    text_from_bytes = binascii.b2a_hex(data.encode("utf-8")).decode()
    len_data = hex(int(len(text_from_bytes)/2))[2:]
    partial_data =  '6a'+ bytes_padding(len_data) + text_from_bytes
    return value_return + bytes_padding(hex(int(len(partial_data)/2))[2:]) + partial_data

In [103]:
def pushdata(len_r,len_s):
    if len_r == '20' and len_s == '20':
        return str(47)    
    elif len_r == '20' or len_s == '21':
        return str(48)
    elif len_r == '21' or len_s == '20':
        return str(48)
    else:
        return str(49)

In [104]:
def pubkey_opcode(a):
    return hex(int(int(len(a))/2))[2:]

In [105]:
print('Hash_160:', hashing(public_key2.hex()))

Hash_160: 7029fd1fedb6aebca27172721c45a1588b70d2e5


In [108]:
#User input:
version_number = '01000000'
number_inputs = '01'
previous_transaction = input("Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: ")
previous_transaction = reverse(previous_transaction)
output_script = '0' + input("Enter previous output (here <= 15), for example, 1: ") + '000000'
script = '1976a914' + input("Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: ") + '88ac'
sequence = 'ffffffff'
tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


#User_output:
Outputs = '02'
out_tx = Outputs + tx_outputs() + op_return() + '0'*8 + '01000000'

#Final Transaction:
raw_transaction = tx_ip + out_tx
print('''Raw transaction: ''', raw_transaction)

#DIGITAL SIGNATURE
unsigned = raw_transaction

temp_bin = binascii.unhexlify(unsigned)
temp_hash = hashlib.sha256(temp_bin).digest()
hash2 = hashlib.sha256(temp_hash).hexdigest()
tx_hash = hashlib.sha256(temp_hash).digest()

private_key = input("Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b28943745b166cc94b30e46c09f778fc: ")
signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

#Generating_SigScript:
DER_input =  str(binascii.hexlify(SIG),'ascii')
header = DER_input[0:2]
Sig_Length = DER_input[2:4]
r_integer = DER_input[4:6]
r_length = DER_input[6:8]
r = DER_input[8:8 + 2 * int(r_length,16)]
s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
s = DER_input[12 + 2 * int(r_length,16):]

pushdata_opcode = pushdata(r_length, s_length)
sighash_code = '01'

publickey = input("Enter Publickey (Compressed or Uncompressed), for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: ")
pubkey_pushdata_opcode = pubkey_opcode(publickey)

final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
SigScript = pubkey_opcode(final_SigScript) + final_SigScript
print("\nSigScript(hex): ",SigScript,"\n\n\n")

#Broadcasting_Transaction:
broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
print('''Transaction to broadcast: ''', broadcasting_tx)

Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: ce137210140c2cc5a5fa040be70a51e3cf98a6e5ab17cb2a3244bb01cb429f77
Enter previous output (here <= 15), for example, 1: 1
Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: 7029fd1fedb6aebca27172721c45a1588b70d2e5
Enter Receiver's Address 1, for example, mp7cUPuKFeR3tm7hMF6Y9PZX89iyHMK67E: mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R
Enter Value in BTC (0.0) to send mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R, for example, 0.009:0.00001
Enter Data to send, for exampe, 'Hello': Nasykhova Anastasia
Raw transaction:  0100000001779f42cb01bb44322acb17abe5a698cfe3510ae70b04faa5c52c0c14107213ce010000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588acffffffff02e8030000000000001976a9146e6494907268b2df0ad95095fc69b36187022fa688ac00000000000000000156a0134e6173796b686f766120416e617374617369610000000001000000
Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b2894

!!!

The message with the full name and surname (Nasykhova Anastasia) is too long

An error occurs when trying to broadcast

I had to shorten the message (entered only the last name: Nasykhova)

In [110]:
#User input:
version_number = '01000000'
number_inputs = '01'
previous_transaction = input("Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: ")
previous_transaction = reverse(previous_transaction)
output_script = '0' + input("Enter previous output (here <= 15), for example, 1: ") + '000000'
script = '1976a914' + input("Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: ") + '88ac'
sequence = 'ffffffff'
tx_ip = version_number + number_inputs + previous_transaction + output_script + script + sequence


#User_output:
Outputs = '02'
out_tx = Outputs + tx_outputs() + op_return() + '0'*8 + '01000000'

#Final Transaction:
raw_transaction = tx_ip + out_tx
print('''Raw transaction: ''', raw_transaction)

#DIGITAL SIGNATURE
unsigned = raw_transaction

temp_bin = binascii.unhexlify(unsigned)
temp_hash = hashlib.sha256(temp_bin).digest()
hash2 = hashlib.sha256(temp_hash).hexdigest()
tx_hash = hashlib.sha256(temp_hash).digest()

private_key = input("Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b28943745b166cc94b30e46c09f778fc: ")
signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)

print("\n\nDigital Signature: ", str(binascii.hexlify(SIG),'ascii'))

#Generating_SigScript:
DER_input =  str(binascii.hexlify(SIG),'ascii')
header = DER_input[0:2]
Sig_Length = DER_input[2:4]
r_integer = DER_input[4:6]
r_length = DER_input[6:8]
r = DER_input[8:8 + 2 * int(r_length,16)]
s_integer = DER_input[8 + 2 * int(r_length,16): 10 + 2 * int(r_length,16)]
s_length = DER_input[10 + 2 * int(r_length,16): 12 + 2 * int(r_length,16)]
s = DER_input[12 + 2 * int(r_length,16):]

pushdata_opcode = pushdata(r_length, s_length)
sighash_code = '01'

publickey = input("Enter Publickey (Compressed or Uncompressed), for example, 0409aa99ce4e7f6b23faf283b5eaab2f6233ac28cbb34cc083957441c5c760fc7b5da64c0975b9f6f8b14f52e1eee5be5750b5ce226b6690bde6438e51211f9ff2: ")
pubkey_pushdata_opcode = pubkey_opcode(publickey)

final_SigScript = pushdata_opcode + header + Sig_Length + r_integer + r_length + r + s_integer + s_length + s + sighash_code + pubkey_pushdata_opcode + publickey
SigScript = pubkey_opcode(final_SigScript) + final_SigScript
print("\nSigScript(hex): ",SigScript,"\n\n\n")

#Broadcasting_Transaction:
broadcasting_tx = version_number + number_inputs + previous_transaction + output_script + SigScript + sequence + out_tx [:-8]
print('''Transaction to broadcast: ''', broadcasting_tx)

Enter Previous user transaction, for example, 0454bef0e9a178b8f31667dc1cb6ba0fb708d4b745d19a8e3ef6de7b52b6108b: ce137210140c2cc5a5fa040be70a51e3cf98a6e5ab17cb2a3244bb01cb429f77
Enter previous output (here <= 15), for example, 1: 1
Enter previous PKH, for example, 76a9145e4f04c3ee1941d665eb1312280ca3a0bbbfd3ba: 7029fd1fedb6aebca27172721c45a1588b70d2e5
Enter Receiver's Address 1, for example, mp7cUPuKFeR3tm7hMF6Y9PZX89iyHMK67E: mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R
Enter Value in BTC (0.0) to send mqaf7bhnZMeTkmy2s2FsDSwmL9WNXTdf3R, for example, 0.009:0.0000001
Enter Data to send, for exampe, 'Hello': Nasykhova
Raw transaction:  0100000001779f42cb01bb44322acb17abe5a698cfe3510ae70b04faa5c52c0c14107213ce010000001976a9147029fd1fedb6aebca27172721c45a1588b70d2e588acffffffff020a000000000000001976a9146e6494907268b2df0ad95095fc69b36187022fa688ac00000000000000000b6a094e6173796b686f76610000000001000000
Enter Private Key, for example, fbed793f3757d1fddf7d675ed64fa329b28943745b166cc94b30e46c09f778fc: e

TX: 7ea57bf8939d97c1ee5d6872537b832d2161bcb7692e710ac242ee025206c442

https://live.blockcypher.com/btc-testnet/tx/7ea57bf8939d97c1ee5d6872537b832d2161bcb7692e710ac242ee025206c442/

### Task 5 (3 points)

From Step 3.2, Make a final transaction to Address n2jmkNzxLJM51AvH3WNyYgWpsrjvR5HnF8

In [112]:
class variableInteger:
    def reverse_string_in_pair(a_str_ip):
        i = 0
        rev_a = []
        while i != len(a_str_ip):
            #print(a[i] + a[i+1])
            temp_value = a_str_ip[i] + a_str_ip[i+1]
            rev_a.append(temp_value)
            i = i + 2
        rev_a.reverse()
        return ''.join(rev_a)
    
    def padding(b):
        if int(len(b) % 2) == 0:
            return b
        else:
            return '0' + b
    
    def variable_integer(a):
        if type(a) == int:
            temp = a
        else:
            temp = int(len(a)/2)
        
        if temp <= int('0xfc',16):
            return variableInteger.padding(hex(temp)[2:])
        elif temp > int('0xfc',16) and temp <= int('0xffff',16):
            return 'fd' +  variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffff',16) and temp <= int('0xffffffff',16):
            return 'fe' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        elif temp > int('0xffffffff',16) and temp <= int('0xffffffffffffffff',16):
            return 'ff' + variableInteger.reverse_string_in_pair(variableInteger.padding(hex(temp)[2:]))
        
def digital_signature(raw_tx):
    unsigned = raw_tx
    
    temp_bin = binascii.unhexlify(unsigned)
    temp_hash = hashlib.sha256(temp_bin).digest()
    tx_hash = hashlib.sha256(temp_hash).digest()
    private_key = input("Enter Private Key: ")
    signingkey = ecdsa.SigningKey.from_string(bytes.fromhex(private_key), curve=ecdsa.SECP256k1)
    SIG = signingkey.sign_digest(tx_hash, sigencode=ecdsa.util.sigencode_der_canonize)   
    temp_sig = str(binascii.hexlify(SIG),'ascii')
    return temp_sig     

def reverse_string_in_pair(a_str_ip):
    i = 0
    rev_a = []
    while i != len(a_str_ip):
        #print(a[i] + a[i+1])
        temp_value = a_str_ip[i] + a_str_ip[i+1]
        rev_a.append(temp_value)
        i = i + 2
    rev_a.reverse()
    return ''.join(rev_a)

def bitcoin_satoshi_rev_hex(btc):
    temp = float(btc) * (10**8)
    temp_1 = hex(int(temp))[2:]
    temp_2 = reverse_string_in_pair(str(temp_1).zfill(16))
    return temp_2

def tx_inputs_tx_id():
    previous_tx_hash = input("Enter Reference TxId: ")
    previous_tx_hash_reversed = reverse_string_in_pair(previous_tx_hash)
    previous_output_index = input("Enter Previous output index: ")
    previous_output_index = reverse_string_in_pair('{:08d}'.format(int(previous_output_index)))
    return previous_tx_hash_reversed + previous_output_index

def redeem_place_holder():
    redeemScript = input("Enter redeem Script: ")
    return variableInteger.variable_integer(redeemScript) + redeemScript

def tx_outputs(c):
    receiver = input("Enter Receiver's Address {}: ".format(c+1))
    if receiver[0] == 'm' or receiver[0] == 'n': 
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = '76' + 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '88' + 'ac'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script
    
    elif receiver[0] == '2':
        value = input("Enter Value in BTC (0.0) to send {}:".format(receiver))
        value_in_hex_NYB = bitcoin_satoshi_rev_hex(value)
        receiver_address = base58.b58decode(receiver).hex()[2:-8]
        locking_Script = 'a9' + variableInteger.variable_integer(receiver_address) + receiver_address + '87'
        script_len_2 = variableInteger.variable_integer(locking_Script)
        return value_in_hex_NYB + script_len_2 + locking_Script

In [113]:
sig_loop = input("Number of Signatures Required: ")
#Making Raw_Transaction: 
#input Basic
version = '01000000'

#input_multi_Signature:
input_ms = '01' + tx_inputs_tx_id()
redeem_Script = redeem_place_holder()
tx_input_value = input_ms + redeem_Script + 'f'*8

Number of Signatures Required: 2
Enter Reference TxId: 10aca83f5341169f9ce257fbd24e9d901b959f74857e00455f9d67c83260f7b4
Enter Previous output index: 1
Enter redeem Script: 524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea9252ae


In [114]:
#output
utxo_output = input("Enter number of outputs: ")
tx_output = ''
for i in range(0, int(utxo_output)):
        tx_output = tx_outputs(i) + tx_output
next_output_data =  variableInteger.variable_integer(int(utxo_output)) + tx_output

locktime = '00000000'
SigHashCode = '01000000'

Enter number of outputs: 1
Enter Receiver's Address 1: n2jmkNzxLJM51AvH3WNyYgWpsrjvR5HnF8
Enter Value in BTC (0.0) to send n2jmkNzxLJM51AvH3WNyYgWpsrjvR5HnF8:0.000001


In [115]:
tx_to_sign = version + tx_input_value + next_output_data + locktime + SigHashCode
print("Raw Transaction to Sign:")
print(tx_to_sign)

Raw Transaction to Sign:
0100000001b4f76032c8679d5f45007e85749f951b909d4ed2fb57e29c9f1641533fa8ac100100000087524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea9252aeffffffff0164000000000000001976a914e8c73eda67ba6bc8b2d52b1d60da9aad9457f6f588ac0000000001000000


In [116]:
signatures = ''

for i in range(0,int(sig_loop)):
    signature_temp = digital_signature(tx_to_sign) + '01'
    signature_temp_2 = variableInteger.variable_integer(signature_temp) + signature_temp
    signatures = signatures + signature_temp_2
    print(i+1,signatures)

Enter Private Key: e9e0825fa7d83dbc2ffd32166c2bf2cb7e77b38d8eab269cc51958e8b91c221e
1 4830450221008b0b396653e1f686a5cd8456f7a1267ea29c2895181c6bc27bdebcb743a5f35102205e39ad4d02fe77abdef9aa30887938124e551c44f39e44e6c08574b53dda6d0501
Enter Private Key: e777808d97db3d835b5ada8b9e9a0661dda6e96d0fdb64f02a8e9642603ddc7c
2 4830450221008b0b396653e1f686a5cd8456f7a1267ea29c2895181c6bc27bdebcb743a5f35102205e39ad4d02fe77abdef9aa30887938124e551c44f39e44e6c08574b53dda6d050148304502210096973334df0c3796809c5b12bdf584afcec5b9bdcd302a13240a177e1c8985bd022058a14676a6c59d302332e1ff5cebb784ce1d5565f561782f6002440e1dbfe2ad01


In [117]:
SigScript_places = '00' + signatures + '4c' + redeem_Script 
SigScript =  variableInteger.variable_integer(SigScript_places) + SigScript_places
print("SigScript:",SigScript)

SigScript: fd1c01004830450221008b0b396653e1f686a5cd8456f7a1267ea29c2895181c6bc27bdebcb743a5f35102205e39ad4d02fe77abdef9aa30887938124e551c44f39e44e6c08574b53dda6d050148304502210096973334df0c3796809c5b12bdf584afcec5b9bdcd302a13240a177e1c8985bd022058a14676a6c59d302332e1ff5cebb784ce1d5565f561782f6002440e1dbfe2ad014c87524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea9252ae


In [118]:
Broadcasting_transaction = version + input_ms + SigScript + 'f'*8 + next_output_data + locktime
print(Broadcasting_transaction)

0100000001b4f76032c8679d5f45007e85749f951b909d4ed2fb57e29c9f1641533fa8ac1001000000fd1c01004830450221008b0b396653e1f686a5cd8456f7a1267ea29c2895181c6bc27bdebcb743a5f35102205e39ad4d02fe77abdef9aa30887938124e551c44f39e44e6c08574b53dda6d050148304502210096973334df0c3796809c5b12bdf584afcec5b9bdcd302a13240a177e1c8985bd022058a14676a6c59d302332e1ff5cebb784ce1d5565f561782f6002440e1dbfe2ad014c87524104847f4086cc82a24ec3625e0d295a08aa7c25334af797a396add7c84f6440ada65247424d39cc15dbf4126c6537be7330ca28b66f7a0e1a5eb22fa9c953972a284104b44be260b27457cd7152c3b13c41046a7af5a21f3c9d99082a7db88a4173410f39990b21b1f905590d5b56a97db0305b0c324372015497b04649ba7439f9ea9252aeffffffff0164000000000000001976a914e8c73eda67ba6bc8b2d52b1d60da9aad9457f6f588ac00000000


TX: 90cd51b2f60dab9e5e1a2c517a49a553cb95a741e362f076f0de818185a299ce

https://live.blockcypher.com/btc-testnet/tx/90cd51b2f60dab9e5e1a2c517a49a553cb95a741e362f076f0de818185a299ce/